# 4.2-a How does the number of classes affect accuracy?

In [1]:
import os
import sys
import datetime
import random
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

import models
import report
from dataset import Dataset
from report import Reporter
from trainer import Trainer
from batch_encoder import Dataset

from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Activation, TimeDistributed, Flatten, Dot, Softmax, Lambda, RepeatVector, Multiply, Permute, Reshape, BatchNormalization

/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
tf.__version__, tf.keras.__version__

('1.14.0', '2.2.4-tf')

In [3]:
tf.test.is_gpu_available(), tf.test.is_built_with_cuda()

(False, False)

In [4]:
raw_dataset_folder='govdocs1/sample200'
minimum=200
maximum=200
result_dir = 'results/4.2a-nclasses'

In [5]:
rawset = Dataset.new_from_folders(raw_dataset_folder)
rawset = rawset.filter(lambda x: rawset.category_from(x) not in ['text', 'unk'])
rawset = rawset.filter_min_max(200, 200)

In [6]:
def CM(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(classes, (32,), strides=1)(last)
    last = MaxPooling1D(pool_size=481, strides=1)(last)
    last = Flatten()(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [7]:
def train_nclasses(rawset, reporter, iterations=1):
    for it in range(iterations):
        for n in range(2, len(rawset.categories)+1):
            cats = random.sample(rawset.by_category().keys(), n)
            print(n, cats)
            myset = rawset.filter(lambda x: rawset.category_from(x) in cats)
            myset.rebuild_categories()
            tset, vset = myset.rnd_split_fraction_by_category(0.5)

            model = CM(n, 256)
            result = Trainer(
                model,
                xs_encoder='one_hot',
                batch_size=100,
                steps_per_epoch=28,
                validation_steps=28,
                patience=10,
            ).train(tset, vset)
            r.line(n=n,
                cats=','.join(cats),
                val_categorical_accuracy=result.history.history['val_categorical_accuracy'][-1],
            )

# Train models

In [ ]:
if os.path.exists(result_dir + "/nclasses.tsv"):
    os.remove(result_dir + "/nclasses.tsv")
r = report.Reporter2(result_dir + '/nclasses.tsv')
train_nclasses(rawset, r, 5)
r.close()

2 ['ps', 'eps']
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
3 ['log', 'rtf', 'xml']
4 ['jpg', 'kml', 'f', 'png']
5 ['doc', 'eps', 'java', 'f', 'rtf']
6 ['pdf', 'pps', 'wp', 'log', 'kmz', 'gz']
7 ['dbase3', 'rtf', 'pdf', 'sql', 'html', 'dwf', 'doc']
8 ['jpg', 'xml', 'gif', 'ppt', 'f', 'rtf', 'dbase3', 'eps']
9 ['png', 'pptx', 'swf', 'log', 'ppt', 'rtf', 'pps', 'html', 'java']
10 ['hlp', 'gz', 'pps', 'gif', 'kmz', 'dwf', 'dbase3', 'java', 'swf', 'doc']
11 ['gif', 'swf', 'kml', 'png', 'doc', 'html', 'dwf', 'hlp', 'xml', 'f', 'eps']
12 ['csv', 'doc', 'ps', 'java', 'html', 'dbase3', 'xls', 'pptx', 'xml', 'kmz', 'kml', 'wp']
13 ['rtf', 'dbase3', 'ps', 'java', 'f', 'pptx', 'txt', 'gz', 'kmz', 'log', 'kml', 'pdf', 'html']
14 ['doc', 'f', 'hlp', 'xls', 'wp', 'jpg', 'java', 'pps', 'eps', 'dbase3', 'gz', 'pptx', 'txt', 'swf']
15 ['rtf', 'gz', 'html', 'dbase3', 'gif', 'java', 'dwf', 'hlp', 'kmz', 'pptx', 'xls', 'doc', 'swf',

# Load data

In [ ]:
data = pd.read_csv(result_dir + '/nclasses.tsv', sep='\t')
data

In [ ]:
from matplotlib.ticker import MultipleLocator
def plot_acc(xs, ys, labels=None, loc='upper right', alpha_on=None):
    fig = plt.figure(figsize=(12,8))
#     fig.yaxis.set_minor_locator()
    if labels is None:
        labels = [None for x in xs]
    for x, y, label, mark in zip(xs, ys, labels, ['bx', 'g-', 'r*', '.mD']):
        plt.plot(x[x==alpha_on], y[x==alpha_on], mark, alpha=0.1, label='')
        plt.plot(x[x!=alpha_on], y[x!=alpha_on], mark, label=label)
    ax1 = plt.gca()
    plt.ylabel('accuracy', fontsize=16)
    plt.xlabel('number of classes', fontsize=16)
    plt.xlim((-0, 31))
    plt.ylim((-0, 1))
    ax1.xaxis.set_major_locator(MultipleLocator(5))
    ax1.xaxis.set_minor_locator(MultipleLocator(1))
    ax1.yaxis.set_major_locator(MultipleLocator(0.1))
    ax1.yaxis.set_minor_locator(MultipleLocator(0.05))
    plt.grid(linestyle='-', color='k', which='major')
    plt.grid(linestyle='--', which='minor')
    for tick in ax1.xaxis.get_major_ticks():
        tick.label.set_fontsize(16) 
    for tick in ax1.yaxis.get_major_ticks():
        tick.label.set_fontsize(16) 
    plt.legend(loc=loc, prop={'size': 16})
#     plt.savefig('acc.png')

In [ ]:
x = np.arange(2,30,0.01)
y = 1/x
plot_acc([data['n'], x], [data['val_categorical_accuracy'], y], ['models trained with many classes', 'random chance'])